In [1]:
import toml
import importlib
from torch.utils.data import DataLoader
import datetime
from PIL import Image
from utils import check_device
import numpy as np
import glob
import os
import torch
from Explanation.evaluation.tools import CausalMetric
device = check_device()
experiment_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
import pickle
adversarial_config = toml.load("Config/adversarial.toml")
explanation_config = toml.load("Config/explanation.toml")
mutants_config = toml.load("Config/mutants.toml")
pruning_config = toml.load("Config/pruning.toml")
pipeline_config = toml.load("Config/pipeline.toml")

In [2]:
def eval_config(config):
    for key in config.keys():
        if isinstance(config[key], dict):
            eval_config(config[key])
        else:
            try:
                config[key] = eval(config[key])
            except:
                pass
    return config

adversarial_config = eval_config(adversarial_config)
explanation_config = eval_config(explanation_config)
mutants_config = eval_config(mutants_config)
pruning_config = eval_config(pruning_config)
pipeline_config = eval_config(pipeline_config)

In [3]:
pipeline_config

{'Config': {'model': 'custom', 'dataset': 'cifar10'},
 'Explanation': {'fast_ig': {'input': 'test_dataloader',
   'save': True,
   'evaluate': True},
  'saliencymap': {'input': 'test_dataloader', 'save': True, 'evaluate': True}}}

In [4]:
model = importlib.import_module(f'Config.Model.{pipeline_config["Config"]["model"]}')
dataset = importlib.import_module(f'Config.Dataset.{pipeline_config["Config"]["dataset"]}')

In [5]:
ComposedModel = model.ComposedModel
NUM_CLASSES = model.NUM_CLASSES
INPUT_SIZE = model.INPUT_SIZE
TYPE = model.TYPE

In [6]:
model = ComposedModel().get_model()

In [7]:
train_dataset,test_dataset = dataset.load_dataset()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
BATCH_SIZE = dataset.BATCH_SIZE
BATCH_SIZE

128

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
del pipeline_config['Config']

In [11]:
pipeline_config

{'Explanation': {'fast_ig': {'input': 'test_dataloader',
   'save': True,
   'evaluate': True},
  'saliencymap': {'input': 'test_dataloader', 'save': True, 'evaluate': True}}}

In [12]:
def mutants_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    function_config = mutants_config.get(f,{})
    function = function(**function_config)
    globals()[cfg["output"]] = DataLoader(input_.dataset,batch_size=BATCH_SIZE,shuffle=False,collate_fn=function)

In [13]:
def adversarial_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    save = cfg["save"]
    function_config = adversarial_config.get(f,{})
    function = function(model,**function_config)
    if f == "advgan":
        function.train(train_dataloader)
    file_name = 0
    all_adversarial_labels = list()
    all_labels = list()
    correct_index = list()
    for x,y in input_:
        x = x.to(device)
        y = y.to(device)
        correct_idx = model(x).argmax(-1) == y
        correct_index.append(correct_idx.cpu().detach().numpy())
        adversarial_images = function(x,y)
        adverserial_labels = model(adversarial_images).argmax(-1).cpu().detach().numpy()
        all_labels.append(y.cpu().detach().numpy())
        if save:
            for adversarial_image in adversarial_images:
                adversarial_image = adversarial_image.cpu().detach().numpy().transpose(1,2,0)
                adversarial_image = Image.fromarray((adversarial_image * 255).astype(np.uint8))
                if not os.path.exists(f"Outputs/{experiment_name}/{module}/{f}"):
                    os.makedirs(f"Outputs/{experiment_name}/{module}/{f}")
                adversarial_image.save(f"Outputs/{experiment_name}/{module}/{f}/{file_name}.png")
                file_name += 1
        all_adversarial_labels.append(adverserial_labels)
    all_adversarial_labels = np.concatenate(all_adversarial_labels)
    all_labels = np.concatenate(all_labels)
    correct_index = np.concatenate(correct_index)
    success_rate = (1 - np.sum(all_adversarial_labels[correct_index] == all_labels[correct_index]) / np.sum(correct_index))
    print(f"{f} Success Rate: {success_rate * 100}%")
    if save:
        result = {
            "all_adversarial_labels":all_adversarial_labels,
            "all_labels":all_labels,
            "success_rate":success_rate
        }
        with open(f"Outputs/{experiment_name}/{module}/{f}/result.pkl","wb") as f:
            pickle.dump(result,f)
    

In [14]:
def explanation_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    save = cfg["save"]
    function_config = explanation_config.get(f,{})
    for i,(x,y) in enumerate(input_):
        x = x.to(device)
        y = y.to(device)
        if f == "fast_ig" or f == "guided_ig":
            attribution = list()
            for x_,y_ in zip(x,y):
                x_ = x_.unsqueeze(0)
                y_ = y_.unsqueeze(0)
                attribution_ = function(model,x_,y_,**function_config)
                attribution.append(attribution_)
            attribution = np.concatenate(attribution)
        else:
            attribution = function(model,x,y,**function_config)
        if save:
            if not os.path.exists(f"Outputs/{experiment_name}/{module}/{f}"):
                os.makedirs(f"Outputs/{experiment_name}/{module}/{f}")
            y = y.cpu().detach().numpy()
            x = x.cpu().detach().numpy()
            np.savez(f"Outputs/{experiment_name}/{module}/{f}/{i}.npz",x=x,y=y,attribution=attribution)
    if cfg['evaluate']:
        results_files = glob.glob(f"Outputs/{experiment_name}/{module}/{f}/*.npz")
        scores = {'del': [], 'ins': []}
        for file in results_files:
            results = np.load(file)
            attribution = results['attribution']
            x = torch.from_numpy(results['x']).to(device)
            deletion = CausalMetric(
                model, 'del', substrate_fn=torch.zeros_like, hw=np.prod(INPUT_SIZE[-2:]), num_classes=NUM_CLASSES)
            insertion = CausalMetric(
                model, 'ins', substrate_fn=torch.zeros_like, hw=np.prod(INPUT_SIZE[-2:]), num_classes=NUM_CLASSES)
            scores['del'].extend(deletion.evaluate(
                x, attribution, len(x)).tolist())
            scores['ins'].extend(insertion.evaluate(
                x, attribution, len(x)).tolist())
        scores['ins'] = np.array(scores['ins'])
        scores['del'] = np.array(scores['del'])
        with open(f'Outputs/{experiment_name}/{module}/{f}/scores.txt', 'w') as f:
            f.write("Insertion: " + str(scores['ins'].mean()) + "\n")
            f.write("Deletion: " + str(scores['del'].mean()) + "\n")
            print("Insertion: " + str(scores['ins'].mean()))
            print("Deletion: " + str(scores['del'].mean()))

In [15]:
def pruning_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    function_config = pruning_config.get(f,{})
    saved_path = f"Outputs/{experiment_name}/{module}/{f}/mask.pkl"
    if not f == "greg":
        function = function(**function_config)
        globals()[f"{f}_pruned_model_pr_{function_config['pruning_ratio']}"] = function(model,train_dataloader)
    else:
        function = function(**function_config)
        globals()[f"{f}_pruned_model_pr_{np.mean(function_config['stage_pr'])}"] = function(model,train_dataloader,test_dataloader)

In [16]:
def distribution(module):
    match module:
        case "Mutants":
            return mutants_runner
        case "Adversarial":
            return adversarial_runner
        case "Pruning":
            return pruning_runner
        case "Explanation":
            return explanation_runner

In [17]:
for module,func in pipeline_config.items():
    for f,cfg in func.items():
        md = importlib.import_module(module)
        function = getattr(md,f)
        print(f"Running {f} from {module}")
        runner = distribution(module)
        runner(module,f,function,cfg)
        

Running fast_ig from Explanation
Insertion: 0.09720203146389395
Deletion: 0.9859139351174235
Running saliencymap from Explanation


c:\Users\Zhang\miniconda3\lib\site-packages\captum\_utils\gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Insertion: 0.09720203146389395
Deletion: 0.9859139351174235


In [1]:
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet50, inception_v3, googlenet, vgg16, mobilenet_v2
from tqdm import tqdm
import torch
import numpy as np
from utils import check_device
device = check_device()
from Explanation.evaluation.tools import CausalMetric
import argparse
import torch
import random
model = inception_v3(pretrained=True).eval().to(device)
sm = nn.Softmax(dim=-1)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
norm_layer = transforms.Normalize(mean, std)
sm = nn.Softmax(dim=-1)
model = nn.Sequential(norm_layer, model, sm).to(device)

c:\Users\Zhang\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zhang\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
deletion = CausalMetric(model, 'del',substrate_fn=torch.zeros_like,hw=224*224,num_classes=1000)
insertion = CausalMetric(model, 'ins',substrate_fn=torch.zeros_like,hw=224*224,num_classes=1000)

In [3]:
img_batch = torch.load("img_batch.pt")

In [4]:
attribution = np.load(f"inception_v3_our_attributions.npy")

In [5]:
scores = {'del': deletion.evaluate(
        img_batch, attribution, 100), 'ins': insertion.evaluate(img_batch, attribution, 100)}

 43%|████▎     | 96/225 [05:36<11:23,  5.30s/it]